## The Challenge

As a wealth manager you have a number of investors who are associated with multiple accounts. These investors can be part of one or more Households, and a Household can contain one or more Investors. You would like to be able to see the holdings for an investor based on each household they are associated with.

## The Solution

1) Create your Instrument universe using a range of identifiers

2) Set up a scope to hold the accounts

3) Create a LUSID Portfolio for each account

4) Create a Property to hold the investor details

5) Set the initial holdings of the portfolio

5) Create a Portfolio Group for each household

6) Load market data (prices)

7) Conduct a valuation against each household

8) Consider other ways of grouping client accounts

In [1]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2744.0


![Scopes](img/paper-lusid.gif)

### Create your instrument universe using a range of identifiers

Before you can take on any holdings for your client accounts you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [2]:
equity_instruments = pd.read_csv('data/households-instruments-equities.csv')
equity_instruments.head(n=20)

InstrumentName ClientInternal Currency  \
0                         Amazon_Nasdaq_AMZN   imd_34634534      USD   
1                          Apple_Nasdaq_AAPL   imd_35345345      USD   
2               VANGUARD LIFESTRATEGY GROWTH   imd_89881022      USD   
3                         Salesforce.com Inc   imd_44953022      USD   
4         Norwegian Cruise Line Holdings Ltd   imd_44569345      USD   
5                           CAMPBELL SOUP CO   imd_44645943      USD   
6                           Under Armour Inc   imd_44678322      USD   
7                             Halliburton Co   imd_44588822      USD   
8                                   MSCI Inc   imd_44999201      USD   
9                          Brown-Forman Corp   imd_45992929      USD   
10            Blackrock Equity Dividend Fund   imd_49302011      USD   
11  T. Rowe Price Instl Small-Cap Stock Fund   imd_44904222      USD   

            Figi ExchangeCode           CountryIssue Ticker MarketSector  \
0   BBG000BVPXP1           UN  united_states_america   AMZN       equity   
1   BBG000B9XVV8           UN  united_states_america   AAPL       equity   
2   BBG000BTB8B1           US  united_states_america  VASGX       equity   
3   BBG000BN2DC2           US  united_states_america    CRM       equity   
4   BBG000BSRN78           US  united_states_america   NCLH       equity   
5   BBG000BG4202           US  united_states_america    CPB       equity   
6   BBG009DTD8H2           US  united_states_america     UA       equity   
7   BBG000BKTFN2           US  united_states_america    HAL       equity   
8   BBG000RTDY25           US  united_states_america   MSCI       equity   
9   BBG000BD2C18           US  united_states_america   BF/A       equity   
10  BBG000BN46T7           US  united_states_america  MDDVX       equity   
11  BBG000BPTDR2           US  united_states_america  TRSSX       equity   

     SecurityType  Coupon  
0    common_stock     NaN  
1    common_stock     NaN  
2   fund_of_funds     NaN  
3    common_stock     NaN  
4    common_stock     NaN  
5    common_stock     NaN  
6    common_stock     NaN  
7    common_stock     NaN  
8    common_stock     NaN  
9    common_stock     NaN  
10  open_end_fund     NaN  
11  open_end_fund     NaN

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [3]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in equity_instruments.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = ['Figi', 'Ticker', 'ClientInternal']
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier] = models.InstrumentIdValue(
            value=instrument[identifier])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
        name=instrument['InstrumentName'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: MSCI Inc
ClientInternal ID: imd_44999201
LUSID Instrument ID: LUID_11JI3OYJ


Instrument Successfully Upserted: Brown-Forman Corp
ClientInternal ID: imd_45992929
LUSID Instrument ID: LUID_RP0IE355


Instrument Successfully Upserted: Blackrock Equity Dividend Fund
ClientInternal ID: imd_49302011
LUSID Instrument ID: LUID_X90EN4H6


Instrument Successfully Upserted: CAMPBELL SOUP CO
ClientInternal ID: imd_44645943
LUSID Instrument ID: LUID_N0ZL2QU8


Instrument Successfully Upserted: T. Rowe Price Instl Small-Cap Stock Fund
ClientInternal ID: imd_44904222
LUSID Instrument ID: LUID_3NL1MMNW


Instrument Successfully Upserted: Under Armour Inc
ClientInternal ID: imd_44678322
LUSID Instrument ID: LUID_M74PQLR3


Instrument Successfully Upserted: Salesforce.com Inc
ClientInternal ID: imd_44953022
LUSID Instrument ID: LUID_P63D2PIX


Instrument Successfully Upserted: Halliburton Co
ClientInternal ID: imd_44588822
LUSID Instrument ID: LUID_KCV3PDG0


Instrumen

![Scopes](img/households-instrumentmaster.gif)

### Set up a scope to hold the accounts

Your wealth managment company has a number of different branches. Each of these branches will be allocated a Scope in LUSID for its accounts. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [4]:
scope_id = import_data.create_scope_id()
scope = 'branch_{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: branch_3770-bede-d0b6-d8


![Scopes](img/households-branch.gif)

### Create a portfolio for each account that exists with the branch

Now that you have decided on the name for your scope, you can create the portfolios to represent the client accounts inside this scope. You will import the account details from a CSV file.

*Run the cell below to import your client's account details*

In [5]:
accounts = pd.read_csv('data/households-accounts.csv')
accounts.head(n=10)

account_code                   account_name currency  \
0  040004-929987648               OneIntlBrokerage      USD   
1  040004-778939522               OneIntlBrokerage      USD   
2  204507-250099511                        VangETF      USD   
3  774001-899930233  FidelityOnlineUSEquityOptions      USD   
4  400250-613060229          ActiveManagedBlckrock      USD   
5  769231-448679901                     MutualFund      USD   

                                       description primary_acount_owner_id  \
0                          Primary trading account        invstr_7325jhv93   
1                                    College fund         invstr_7325jhv93   
2                     Passive exchange traded fund        invstr_a1kf37761   
3                 Online US equity & option trades        invstr_bbgj93921   
4  Actively managed high return investment account        invstr_a1kf37761   
5                        Pooled investment vehicle        invstr_7325jhv93   

     household_id other_account_owner_id financial_advisor_id  
0  hhd_jxgru45055                    NaN      ffid_abkjas9932  
1  hhd_jxgru45055       invstr_bbgj93921      ffid_abkjas9932  
2   hhd_ab3452342                    NaN      ffid_abkjas9932  
3  hhd_jxgru45055                    NaN      ffid_xxye990221  
4   hhd_ab3452342                    NaN      ffid_abkjas9932  
5   hhd_ab3452342                    NaN      ffid_abkjas9932

With the account details loaded you can now create your client portfolios.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [6]:
# Iterate over the client accounts
for index, account in accounts.iterrows():

    # Set the creation date of your client portfolio 
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

    # Build your request to create your client portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account['account_name'],
        code=account['account_code'],
        base_currency=account['currency'],
        description=account['description'],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=None,
        properties=None)

    # Call LUSID to create your client portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 040004-929987648
Portfolio Effective From: 2016-07-27 10:41:20.401944+00:00
Portfolio Created On: 2019-06-14 10:41:18.755252+00:00

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 040004-778939522
Portfolio Effective From: 2016-07-27 10:41:20.702743+00:00
Portfolio Created On: 2019-06-14 10:41:19.004260+00:00

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 204507-250099511
Portfolio Effective From: 2016-07-27 10:41:20.948400+00:00
Portfolio Created On: 2019-06-14 10:41:19.231996+00:00

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 774001-899930233
Portfolio Effective From: 2016-07-27 10:41:21.402830+00:00
Portfolio Created On: 2019-06-14 10:41:19.702658+00:00

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 400250-613060229
Portfolio Effective From: 2016-07-27 10:41:21.657212+00:00
Portfolio Created On: 2019-06-14 10:41:19.931120+00:00

Portfolio Created
Scope: branch_3770-bede-d0b6-d8
Code: 7692

![Scopes](img/households-portfolio.gif)

### Create a property to hold the investor and the household

To keep track of the investor associated with each portfolio as well as the household you can make use of LUSID's extensible properties. These allow you to define a bespoke schema for your portfolio objects. Read more about properties in the [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold household_id, the account owner ids & the financial advisor id*

In [7]:
# The property codes to create
property_codes = [
    'household_id', 
    'primary_acount_owner_id', 
    'other_account_owner_id',
    'financial_advisor_id']

# Iterate over the property codes
for property_code in property_codes:

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='default', code='string'))

    # Call LUSID to create your new property
    response = client.property_definitions.create_property_definition(
        definition=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    portfolio_property_key = response.key

    # Pretty print your key
    prettyprint.heading('Portfolio Property Key', portfolio_property_key)

Portfolio Property Key: Portfolio/branch_3770-bede-d0b6-d8/household_id
Portfolio Property Key: Portfolio/branch_3770-bede-d0b6-d8/primary_acount_owner_id
Portfolio Property Key: Portfolio/branch_3770-bede-d0b6-d8/other_account_owner_id
Portfolio Property Key: Portfolio/branch_3770-bede-d0b6-d8/financial_advisor_id


With these properties created you can now populate them for each portfolio.

*Run the cell below to populate the values for these properties for each portfolio*

In [8]:
# Iterate over each account
for index, account in accounts.iterrows():
    
    # Add the relevant account properties to the portfolio
    response = client.portfolios.upsert_portfolio_properties(
        scope=scope,
        code=account['account_code'],
        portfolio_properties={
            'Portfolio/{}/{}'.format(scope, account_property): models.PropertyValue(
                label_value=account[account_property]
            ) for account_property in property_codes
        }
    )

    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    print ('\n')

Properties Sucessfully Updated for Portfolio
Scope:  branch_3770-bede-d0b6-d8
Code:  040004-929987648 

Property key: Portfolio/branch_3770-bede-d0b6-d8/other_account_owner_id
Value: NaN

Property key: Portfolio/branch_3770-bede-d0b6-d8/primary_acount_owner_id
Value: invstr_7325jhv93

Property key: Portfolio/branch_3770-bede-d0b6-d8/financial_advisor_id
Value: ffid_abkjas9932

Property key: Portfolio/branch_3770-bede-d0b6-d8/household_id
Value: hhd_jxgru45055



Properties Sucessfully Updated for Portfolio
Scope:  branch_3770-bede-d0b6-d8
Code:  040004-778939522 

Property key: Portfolio/branch_3770-bede-d0b6-d8/other_account_owner_id
Value: invstr_bbgj93921

Property key: Portfolio/branch_3770-bede-d0b6-d8/primary_acount_owner_id
Value: invstr_7325jhv93

Property key: Portfolio/branch_3770-bede-d0b6-d8/financial_advisor_id
Value: ffid_abkjas9932

Property key: Portfolio/branch_3770-bede-d0b6-d8/household_id
Value: hhd_jxgru45055



Properties Sucessfully Updated for Portfolio
Scope:  

![Scopes](img/households-properties.gif)

### Set the initial holdings of the portfolio

Now that you have your instrument universe populated and portfolios created you can load your current client holdings into their portfolios. In this case you will import their holdings from a CSV file. 

*Run the cell below to import your current client holdings*

In [9]:
holdings = pd.read_csv('data/households-holdings.csv')
holdings.head(n=20)

portfolio_code                     instrument_name  quantity    price  \
0   040004-929987648                            USD_Cash     25000     1.00   
1   040004-929987648                  Amazon_Nasdaq_AMZN        31  1538.50   
2   040004-929987648                  Salesforce.com Inc        52   152.49   
3   040004-929987648  Norwegian Cruise Line Holdings Ltd        80    68.98   
4   040004-929987648                    CAMPBELL SOUP CO       150    44.25   
5   040004-929987648                    Under Armour Inc       300    22.52   
6   040004-929987648                   Apple_Nasdaq_AAPL        60   182.19   
7   040004-778939522                            USD_Cash     15000     1.00   
8   040004-778939522                   Brown-Forman Corp       250    59.86   
9   040004-778939522                      Halliburton Co        39    18.54   
10  040004-778939522                  Salesforce.com Inc       100   160.79   
11  040004-778939522                  Amazon_Nasdaq_AMZN         8  1487.92   
12  204507-250099511        VANGUARD LIFESTRATEGY GROWTH      3250    32.80   
13  204507-250099511                            USD_Cash      1523     1.00   
14  774001-899930233                    Under Armour Inc       300    22.99   
15  774001-899930233                      Halliburton Co       300    17.56   
16  774001-899930233                            MSCI Inc       155   234.50   
17  774001-899930233                            USD_Cash     25987     1.00   
18  400250-613060229      Blackrock Equity Dividend Fund       523    20.68   
19  400250-613060229                            USD_Cash      2800     1.00   

   currency          figi  
0       USD           NaN  
1       USD  BBG000BVPXP1  
2       USD  BBG000BN2DC2  
3       USD  BBG000BSRN78  
4       USD  BBG000BG4202  
5       USD  BBG009DTD8H2  
6       USD  BBG000B9XVV8  
7       USD           NaN  
8       USD  BBG000BD2C18  
9       USD  BBG000BKTFN2  
10      USD  BBG000BN2DC2  
11      USD  BBG000BVPXP1  
12      USD  BBG000BTB8B1  
13      USD           NaN  
14      USD  BBG009DTD8H2  
15      USD  BBG000BKTFN2  
16      USD  BBG000RTDY25  
17      USD           NaN  
18      USD  BBG000BN46T7  
19      USD           NaN

Now that you have imported your client holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [10]:
# Make the holdings effective from now
holdings_effective_date = datetime.now(pytz.UTC)

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = client.transaction_portfolios.set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

Holdings Successfully Set for Portfolio
Scope: branch_3770-bede-d0b6-d8
Code: 040004-929987648
Holdings Effective From: 2019-06-14 10:41:26.699295+00:00
Holdings Created On: 2019-06-14 10:41:25.296937+00:00

Holdings Successfully Set for Portfolio
Scope: branch_3770-bede-d0b6-d8
Code: 040004-778939522
Holdings Effective From: 2019-06-14 10:41:26.699295+00:00
Holdings Created On: 2019-06-14 10:41:25.807241+00:00

Holdings Successfully Set for Portfolio
Scope: branch_3770-bede-d0b6-d8
Code: 204507-250099511
Holdings Effective From: 2019-06-14 10:41:26.699295+00:00
Holdings Created On: 2019-06-14 10:41:26.322649+00:00

Holdings Successfully Set for Portfolio
Scope: branch_3770-bede-d0b6-d8
Code: 774001-899930233
Holdings Effective From: 2019-06-14 10:41:26.699295+00:00
Holdings Created On: 2019-06-14 10:41:26.863702+00:00

Holdings Successfully Set for Portfolio
Scope: branch_3770-bede-d0b6-d8
Code: 400250-613060229
Holdings Effective From: 2019-06-14 10:41:26.699295+00:00
Holdings Create

### Create a portfolio group for each household

Now that you've created Portfolios for the client accounts, you can group them together into households using Portfolio Groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a portfolio group for each household*

In [11]:
# Iterate over all households
for household in accounts['household_id'].unique():
    
    # Build a create group request for this group
    group_request = models.CreatePortfolioGroupRequest(
    code=household+'-Group',
    display_name='Contains all accounts for the {} household'.format(household))

    # Call LUSID to create the portfolio group for the household
    response = client.portfolio_groups.create_portfolio_group(
        scope=scope,
        request=group_request)

    # Pretty print the response
    prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Contains all accounts for the hhd_jxgru45055 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the hhd_ab3452342 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 




You can now add the portfolios for each household to the appropriate group.

*Run the cell below to add all of the portfolios in each household to the group*

In [12]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # For this portfolio get its properties which you defined earlier
    properties = client.portfolios.get_portfolio_properties(
        scope=portfolio.id.scope, 
        code=portfolio.id.code)
    
    # Make the list of properties easy to work with by converting them to key-value pairs
    portfolio_properties = {prop.key: prop.value for prop in properties.properties}
    
    # Using the household_id property determine which portfolio group to add this portfolio too
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code=portfolio_properties['Portfolio/{}/household_id'.format(scope)]+'-Group',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Contains all accounts for the hhd_ab3452342 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
400250-613060229
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_ab3452342 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
400250-613060229
769231-448679901
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_jxgru45055 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
774001-899930233
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_ab3452342 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
400250-613060229
769231-448679901
204507-250099511
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_jxgru45055 household
Scope: branch_3770-bede-d0b6-d8
Code: hhd_jx

![Scopes](img/households-portfoliogroupshouseholds.gif)

### Load market data prices

With the Portfolio Groups created to group each household together, you can now aggregate across households.

To aggregate & value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation).

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [13]:
# Import the market data prices
prices = pd.read_csv('data/households-prices.csv')
prices.head(n=50)

instrument_name currency          figi  \
0               VANGUARD LIFESTRATEGY GROWTH      USD  BBG000BTB8B1   
1                         Amazon_Nasdaq_AMZN      USD  BBG000BVPXP1   
2                          Apple_Nasdaq_AAPL      USD  BBG000B9XVV8   
3                         Salesforce.com Inc      USD  BBG000BN2DC2   
4         Norwegian Cruise Line Holdings Ltd      USD  BBG000BSRN78   
5                           CAMPBELL SOUP CO      USD  BBG000BG4202   
6                           Under Armour Inc      USD  BBG009DTD8H2   
7                             Halliburton Co      USD  BBG000BKTFN2   
8                                   MSCI Inc      USD  BBG000RTDY25   
9                          Brown-Forman Corp      USD  BBG000BD2C18   
10            Blackrock Equity Dividend Fund      USD  BBG000BN46T7   
11  T. Rowe Price Instl Small-Cap Stock Fund      USD  BBG000BPTDR2   

   price_current ticker client_internal  
0           33.2  VASGX    imd_34634534  
1       1,738.50   AMZN    imd_35345345  
2         182.54   AAPL    imd_89881022  
3         158.44    CRM    imd_44953022  
4          52.38   NCLH    imd_44569345  
5          41.93    CPB    imd_44645943  
6          25.76     UA    imd_44678322  
7          21.19    HAL    imd_44588822  
8          234.5   MSCI    imd_44999201  
9           54.1   BF/A    imd_45992929  
10         20.72  MDDVX    imd_49302011  
11         24.19  TRSSX    imd_44904222

Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [14]:
# Initialise an empty list to hold the market data quotes
instrument_quotes = []

# Iterate over each quote
for index, quote in prices.iterrows():
    
    # Get the LUSID Instrument ID for the quoted instrument
    luid = client.search.instruments_search(
        symbols=[
            models.InstrumentSearchProperty(
                key='Instrument/default/Figi',
                value=quote['figi'])
        ],
        mastered_only=True
        )[0].mastered_instruments[0].identifiers['LusidInstrumentId'].value
    
    # Create a quote for this instrument and append it to the list of quotes
    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            provider='DataScope',
            price_source='',
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        effective_at=holdings_effective_date,
        lineage='InternalSystem'
    ))

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-06-14 10:41:33.719087+00:00


### Conduct a valuation against each household

Now that the quotes have been added to the quote store you can aggregate across each household. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value each household's investments*

In [15]:
# Create an inline recipe to configue the aggregation logic
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='LusidInstrumentId',
            default_scope=scope)
        )
    )

aggregation_results_df = []

# Iterate over each household
for household in accounts['household_id'].unique():
    
    # Create the aggregation request
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Sum'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value')
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    # Call LUSID to perform an aggregation
    response = client.aggregation.get_aggregation_by_group(
        scope=scope,
        code=household+'-Group',
        request=aggregation_request)

    # Pretty print the response
    aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=household))

In [16]:
prettyprint.heading('Household: ', aggregation_results_df[0].name)
aggregation_results_df[0]

Household: : hhd_jxgru45055


Holding/default/SubHoldingKey  \
Instrument/default/Name                                                   
Amazon_Nasdaq_AMZN                  LusidInstrumentId=LUID_ZJNWLTOX/USD   
Apple_Nasdaq_AAPL                   LusidInstrumentId=LUID_PAXBX2NQ/USD   
Brown-Forman Corp                   LusidInstrumentId=LUID_RP0IE355/USD   
CAMPBELL SOUP CO                    LusidInstrumentId=LUID_N0ZL2QU8/USD   
Cash                                                       Currency=USD   
Halliburton Co                      LusidInstrumentId=LUID_KCV3PDG0/USD   
MSCI Inc                            LusidInstrumentId=LUID_11JI3OYJ/USD   
Norwegian Cruise Line Holdings Ltd  LusidInstrumentId=LUID_USFQI3JA/USD   
Salesforce.com Inc                  LusidInstrumentId=LUID_P63D2PIX/USD   
Under Armour Inc                    LusidInstrumentId=LUID_M74PQLR3/USD   
TOTAL                                                               NaN   

                                    Sum(Holding/default/Cost)  \
Instrument/default/Name                                         
Amazon_Nasdaq_AMZN                                  59,596.86   
Apple_Nasdaq_AAPL                                    10,931.4   
Brown-Forman Corp                                    14,965.0   
CAMPBELL SOUP CO                                      6,637.5   
Cash                                                 65,987.0   
Halliburton Co                                       5,991.06   
MSCI Inc                                             36,347.5   
Norwegian Cruise Line Holdings Ltd                    5,518.4   
Salesforce.com Inc                                  24,008.48   
Under Armour Inc                                     13,653.0   
TOTAL                                               243,636.2   

                                    Sum(Holding/default/PV)  \
Instrument/default/Name                                       
Amazon_Nasdaq_AMZN                                 67,801.5   
Apple_Nasdaq_AAPL                                  10,952.4   
Brown-Forman Corp                                  13,525.0   
CAMPBELL SOUP CO                                    6,289.5   
Cash                                               65,987.0   
Halliburton Co                                     7,183.41   
MSCI Inc                                           36,347.5   
Norwegian Cruise Line Holdings Ltd                  4,190.4   
Salesforce.com Inc                                24,082.88   
Under Armour Inc                                   15,456.0   
TOTAL                                            251,815.59   

                                    Sum(Holding/default/Units)  
Instrument/default/Name                                         
Amazon_Nasdaq_AMZN                                        39.0  
Apple_Nasdaq_AAPL                                         60.0  
Brown-Forman Corp                                        250.0  
CAMPBELL SOUP CO                                         150.0  
Cash                                                  65,987.0  
Halliburton Co                                           339.0  
MSCI Inc                                                 155.0  
Norwegian Cruise Line Holdings Ltd                        80.0  
Salesforce.com Inc                                       152.0  
Under Armour Inc                                         600.0  
TOTAL                                                 67,812.0

In [17]:
prettyprint.heading('Household: ', aggregation_results_df[1].name)
aggregation_results_df[1]

Household: : hhd_ab3452342


Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_X90EN4H6/USD   
Cash                                                             Currency=USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_3NL1MMNW/USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_L3SFZRG1/USD   
TOTAL                                                                     NaN   

                                          Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Blackrock Equity Dividend Fund                            10,815.64   
Cash                                                        6,523.0   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                           137,493.91999999998   

                                          Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Blackrock Equity Dividend Fund                          10,836.56   
Cash                                                      6,523.0   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                   138,709.2   

                                          Sum(Holding/default/Units)  
Instrument/default/Name                                               
Blackrock Equity Dividend Fund                                 523.0  
Cash                                                         6,523.0  
T. Rowe Price Instl Small-Cap Stock Fund                       556.0  
VANGUARD LIFESTRATEGY GROWTH                                 3,250.0  
TOTAL                                                       10,852.0

Perhaps you might also like to group your client accounts by financial advisor. Using Portfolio Groups you can do this as well.

*Run the cell below to create a Portfolio Group for each advisor*

In [18]:
# Iterate over all financial advisors
for advisor in accounts['financial_advisor_id'].unique():
    
    # Build a create group request for this advisor
    group_request = models.CreatePortfolioGroupRequest(
    code=advisor+'-Group',
    display_name='Contains all accounts for the financial advisor with id {}'.format(advisor))

    # Call LUSID to create the portfolio group for the advisor
    response = client.portfolio_groups.create_portfolio_group(
        scope=scope,
        request=group_request)

    # Pretty print the response
    prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Contains all accounts for the financial advisor with id ffid_abkjas9932
Scope: branch_3770-bede-d0b6-d8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the financial advisor with id ffid_xxye990221
Scope: branch_3770-bede-d0b6-d8
Code: ffid_xxye990221-Group
Portfolios Inside Group: 




You can now add the portfolios for each advisor to the appropriate group.

*Run the cell below to add all of the portfolios in for each advisor to the appropriate group*

In [19]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # For this portfolio get its properties which you defined earlier
    properties = client.portfolios.get_portfolio_properties(
        scope=portfolio.id.scope, 
        code=portfolio.id.code)
    
    # Make the list of properties easy to work with by converting them to key-value pairs
    portfolio_properties = {prop.key: prop.value for prop in properties.properties}
    
    # Using the financial_advisor_id property determine which portfolio group 
    # to add this portfolio too
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code=portfolio_properties['Portfolio/{}/financial_advisor_id'.format(scope)]+'-Group',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Contains all accounts for the financial advisor with id ffid_abkjas9932
Scope: branch_3770-bede-d0b6-d8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 
400250-613060229
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial advisor with id ffid_abkjas9932
Scope: branch_3770-bede-d0b6-d8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 
400250-613060229
769231-448679901
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial advisor with id ffid_xxye990221
Scope: branch_3770-bede-d0b6-d8
Code: ffid_xxye990221-Group
Portfolios Inside Group: 
774001-899930233
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial advisor with id ffid_abkjas9932
Scope: branch_3770-bede-d0b6-d8
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 
400250-613060229
769231-448679901
204507-250099511
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts

![Scopes](img/households-portfoliogroupsadvisor.gif)

Now you can aggregate by financial advisor.

*Run the cell below to aggregate your client accounts by financial advisor*

In [20]:
aggregation_results_df = []

# Iterate over each household
for advisor in accounts['financial_advisor_id'].unique():
    
    # Create the aggregation request
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Sum'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value')
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    # Call LUSID to perform an aggregation
    response = client.aggregation.get_aggregation_by_group(
        scope=scope,
        code=advisor+'-Group',
        request=aggregation_request)

    # Append the response to the list of responses
    aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=advisor))

In [21]:
prettyprint.heading('Financial Advisor: ', aggregation_results_df[0].name)
aggregation_results_df[0]

Financial Advisor: : ffid_abkjas9932


Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_ZJNWLTOX/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_PAXBX2NQ/USD   
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_X90EN4H6/USD   
Brown-Forman Corp                         LusidInstrumentId=LUID_RP0IE355/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_N0ZL2QU8/USD   
Cash                                                             Currency=USD   
Halliburton Co                            LusidInstrumentId=LUID_KCV3PDG0/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_USFQI3JA/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_P63D2PIX/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_3NL1MMNW/USD   
Under Armour Inc                          LusidInstrumentId=LUID_M74PQLR3/USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_L3SFZRG1/USD   
TOTAL                                                                     NaN   

                                          Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                        59,596.86   
Apple_Nasdaq_AAPL                                          10,931.4   
Blackrock Equity Dividend Fund                            10,815.64   
Brown-Forman Corp                                          14,965.0   
CAMPBELL SOUP CO                                            6,637.5   
Cash                                                       46,523.0   
Halliburton Co                                               723.06   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                        24,008.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
Under Armour Inc                                            6,756.0   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                                    306,630.62   

                                          Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       67,801.5   
Apple_Nasdaq_AAPL                                        10,952.4   
Blackrock Equity Dividend Fund                          10,836.56   
Brown-Forman Corp                                        13,525.0   
CAMPBELL SOUP CO                                          6,289.5   
Cash                                                     46,523.0   
Halliburton Co                                             826.41   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                      24,082.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
Under Armour Inc                                          7,728.0   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                  314,105.29   

                                          Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              39.0  
Apple_Nasdaq_AAPL                                               60.0  
Blackrock Equity Dividend Fund                                 523.0  
Brown-Forman Corp                                              250.0  
CAMPBELL SOUP CO                                               150.0  
Cash                                                        46,523.0  
Halliburton Co                                                  39.0  
Norwegian Cruise Line Holdings Ltd                              80.0  
Salesforce.com Inc  

In [22]:
prettyprint.heading('Financial Advisor: ', aggregation_results_df[1].name)
aggregation_results_df[1]

Financial Advisor: : ffid_xxye990221


Holding/default/SubHoldingKey  \
Instrument/default/Name                                        
Cash                                            Currency=USD   
Halliburton Co           LusidInstrumentId=LUID_KCV3PDG0/USD   
MSCI Inc                 LusidInstrumentId=LUID_11JI3OYJ/USD   
Under Armour Inc         LusidInstrumentId=LUID_M74PQLR3/USD   
TOTAL                                                    NaN   

                         Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
Instrument/default/Name                                                       
Cash                                      25,987.0                 25,987.0   
Halliburton Co                             5,268.0                  6,357.0   
MSCI Inc                                  36,347.5                 36,347.5   
Under Armour Inc                           6,897.0                  7,728.0   
TOTAL                                     74,499.5                 76,419.5   

                         Sum(Holding/default/Units)  
Instrument/default/Name                              
Cash                                       25,987.0  
Halliburton Co                                300.0  
MSCI Inc                                      155.0  
Under Armour Inc                              300.0  
TOTAL                                      26,742.0

You can also create a portfolio group to see the investment across all the client accounts.

*Run the cell below to create a portfolio group for all accounts*

In [23]:
# Build a create group request for this group
group_request = models.CreatePortfolioGroupRequest(
code='AllAccounts',
display_name='Contains all client accounts for {}'.format(scope))

# Call LUSID to create the portfolio group for all client accounts
response = client.portfolio_groups.create_portfolio_group(
    scope=scope,
    request=group_request)

# Pretty print the response
prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: branch_3770-bede-d0b6-d8
Code: AllAccounts
Portfolios Inside Group: 




In [24]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # Add the portfolio the all accounts portfolio group
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code='AllAccounts',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: branch_3770-bede-d0b6-d8
Code: AllAccounts
Portfolios Inside Group: 
400250-613060229
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: branch_3770-bede-d0b6-d8
Code: AllAccounts
Portfolios Inside Group: 
400250-613060229
769231-448679901
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: branch_3770-bede-d0b6-d8
Code: AllAccounts
Portfolios Inside Group: 
400250-613060229
769231-448679901
774001-899930233
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: branch_3770-bede-d0b6-d8
Code: AllAccounts
Portfolios Inside Group: 
400250-613060229
769231-448679901
774001-899930233
204507-250099511
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all client accounts for branch_3770-bede-d0b6-d8
Scope: b

![Scopes](img/households-portfoliogroupsallaccounts.gif)

You can now aggregate across all client accounts.

*Run the cell below to aggregate across all client accounts*

In [25]:
aggregation_results_df = []

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value')
    ],
    group_by=[
        'Instrument/default/Name'
    ])

# Call LUSID to perform an aggregation
response = client.aggregation.get_aggregation_by_group(
    scope=scope,
    code='AllAccounts',
    request=aggregation_request)

# Add the response to the list
aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=scope))

In [26]:
prettyprint.heading('Branch: ', aggregation_results_df[0].name)
aggregation_results_df[0]

Branch: : branch_3770-bede-d0b6-d8


Holding/default/SubHoldingKey  \
Instrument/default/Name                                                         
Amazon_Nasdaq_AMZN                        LusidInstrumentId=LUID_ZJNWLTOX/USD   
Apple_Nasdaq_AAPL                         LusidInstrumentId=LUID_PAXBX2NQ/USD   
Blackrock Equity Dividend Fund            LusidInstrumentId=LUID_X90EN4H6/USD   
Brown-Forman Corp                         LusidInstrumentId=LUID_RP0IE355/USD   
CAMPBELL SOUP CO                          LusidInstrumentId=LUID_N0ZL2QU8/USD   
Cash                                                             Currency=USD   
Halliburton Co                            LusidInstrumentId=LUID_KCV3PDG0/USD   
MSCI Inc                                  LusidInstrumentId=LUID_11JI3OYJ/USD   
Norwegian Cruise Line Holdings Ltd        LusidInstrumentId=LUID_USFQI3JA/USD   
Salesforce.com Inc                        LusidInstrumentId=LUID_P63D2PIX/USD   
T. Rowe Price Instl Small-Cap Stock Fund  LusidInstrumentId=LUID_3NL1MMNW/USD   
Under Armour Inc                          LusidInstrumentId=LUID_M74PQLR3/USD   
VANGUARD LIFESTRATEGY GROWTH              LusidInstrumentId=LUID_L3SFZRG1/USD   
TOTAL                                                                     NaN   

                                          Sum(Holding/default/Cost)  \
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                        59,596.86   
Apple_Nasdaq_AAPL                                          10,931.4   
Blackrock Equity Dividend Fund                            10,815.64   
Brown-Forman Corp                                          14,965.0   
CAMPBELL SOUP CO                                            6,637.5   
Cash                                                       72,510.0   
Halliburton Co                                             5,991.06   
MSCI Inc                                                   36,347.5   
Norwegian Cruise Line Holdings Ltd                          5,518.4   
Salesforce.com Inc                                        24,008.48   
T. Rowe Price Instl Small-Cap Stock Fund                  13,555.28   
Under Armour Inc                                           13,653.0   
VANGUARD LIFESTRATEGY GROWTH                              106,600.0   
TOTAL                                                    381,130.12   

                                          Sum(Holding/default/PV)  \
Instrument/default/Name                                             
Amazon_Nasdaq_AMZN                                       67,801.5   
Apple_Nasdaq_AAPL                                        10,952.4   
Blackrock Equity Dividend Fund                          10,836.56   
Brown-Forman Corp                                        13,525.0   
CAMPBELL SOUP CO                                          6,289.5   
Cash                                                     72,510.0   
Halliburton Co                                           7,183.41   
MSCI Inc                                                 36,347.5   
Norwegian Cruise Line Holdings Ltd                        4,190.4   
Salesforce.com Inc                                      24,082.88   
T. Rowe Price Instl Small-Cap Stock Fund                13,449.64   
Under Armour Inc                                         15,456.0   
VANGUARD LIFESTRATEGY GROWTH                            107,900.0   
TOTAL                                                  390,524.79   

                                          Sum(Holding/default/Units)  
Instrument/default/Name                                               
Amazon_Nasdaq_AMZN                                              39.0  
Apple_Nasdaq_AAPL                                               60.0  
Blackrock Equity Dividend Fund                                 523.0  
Brown-Forman Corp                                              250.0  
CAMPBELL SOUP CO                                               150.0  
Cash        